In [1]:
import tritonclient.http as httpclient
import numpy as np

In [2]:
client = httpclient.InferenceServerClient(url="localhost:8000")

In [3]:
# test connection
print("client.is_server_live():", client.is_server_live())

client.is_server_live(): True


In [4]:
input_data = np.random.rand(1, 3, 640, 640).astype(np.float32)

In [7]:
inputs = httpclient.InferInput("images", input_data.shape, "FP32")
inputs.set_data_from_numpy(input_data)

In [31]:
output0 = httpclient.InferRequestedOutput("output0")
output1 = httpclient.InferRequestedOutput("output1")

In [32]:
response = client.infer(model_name="yolo", inputs=[inputs], outputs=[output0, output1])

In [35]:
output_data0 = response.as_numpy("output0")
output_data1 = response.as_numpy("output1")

In [50]:
print("output_data0.shape:", output_data0.shape)
print("output_data1.shape:", output_data1.shape)

output_data0.shape: (1, 37, 8400)
output_data1.shape: (1, 32, 160, 160)


In [56]:
import cv2

In [72]:
conf_threshold = 0
iou_threshold = 0.1
im0 = input_data.squeeze().transpose(1, 2, 0)
ratio = (640, 640)
pad_w = 0
pad_h = 0
nm = 32

x, protos = output_data0, output_data1
x = np.einsum("bcn->bnc", x)
x = x[np.amax(x[..., 4:-nm], axis=-1) > conf_threshold]
x = np.c_[x[..., :4], np.amax(x[..., 4:-nm], axis=-1), np.argmax(x[..., 4:-nm], axis=-1), x[..., -nm:]]
x = x[cv2.dnn.NMSBoxes(x[:, :4], x[:, 4], conf_threshold, iou_threshold)]

In [73]:
x

array([[ 6.04696045e+02,  6.29812744e+02,  6.85888672e+01, ...,
        -1.99246064e-01, -2.06948370e-02,  8.65826383e-03],
       [ 5.86759827e+02,  5.74778381e+02,  1.05589355e+02, ...,
        -3.44821483e-01, -1.49340048e-01,  3.11803594e-02],
       [ 3.80890541e+01,  6.14234131e+02,  7.69621201e+01, ...,
        -3.36808771e-01, -5.84300458e-02,  1.37323588e-02],
       ...,
       [ 2.16674469e+02,  5.59357239e+02,  4.20315094e+01, ...,
        -4.21015263e-01, -6.71976447e-01, -7.82849193e-02],
       [ 4.14765076e+02,  5.59530151e+02,  4.50600281e+01, ...,
        -4.41726267e-01, -6.82584941e-01, -7.88767934e-02],
       [ 5.59885330e+01,  5.52381531e+02,  4.55781441e+01, ...,
        -4.84497726e-01, -6.80336952e-01, -1.26507968e-01]])

In [ ]:
x = x[np.amax(x[..., 4:-nm], axis=-1) > conf_threshold]

(1, 8400, 37)

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def get_mask(row, box, img_w, img_h):
    mask = row.reshape(160, 160)
    mask = sigmoid(mask)
    mask = (mask > 0.5).astype('uint8') * 255

    # Calculate the bounding box coordinates in the mask's scale
    mask_x1 = round(box.x1 / img_w * 160)
    mask_y1 = round(box.y1 / img_h * 160)
    mask_x2 = round(box.x2 / img_w * 160)
    mask_y2 = round(box.y2 / img_h * 160)

    # Crop the mask to the bounding box
    mask = mask[mask_y1:mask_y2, mask_x1:mask_x2]

    # Resize mask directly to the bounding box size
    desired_width = round(box.x2 - box.x1)
    desired_height = round(box.y2 - box.y1)
    mask_resized = cv2.resize(mask, (desired_width, desired_height), interpolation=cv2.INTER_LINEAR)

    return mask_resized